In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator

from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.ml.classification import LogisticRegression


In [2]:
IS_SPARK_SUBMIT_CLI = False
if IS_SPARK_SUBMIT_CLI:
    sc = SparkContext.getOrCreate()
    spark = SparkSession(sc)

In [3]:
if IS_SPARK_SUBMIT_CLI:
    coviddeath = spark.read.csv('uscasestemp1.csv', inferSchema=True, header=True)
else:
    coviddeath = spark.sql("SELECT * FROM uscasestemp1_csv")

In [4]:
data = coviddeath.select("Year","Date","Date1","Day", "Temp", "Admin2","Lat","Long","Province",col("Case").alias("label"))
data = StringIndexer(inputCol='Admin2', outputCol='Admin2'+"_index").fit(data).transform(data)
data = StringIndexer(inputCol='Province', outputCol='Province'+"_index").fit(data).transform(data)
data = StringIndexer(inputCol='Date1', outputCol='Date1'+"_index").fit(data).transform(data)

data.show(5)


+----+----+--------+---+----+-----------+-----------+------------+----------+-----+------------+--------------+-----------+
Year|Date| Date1|Day|Temp| Admin2| Lat| Long| Province|label|Admin2_index|Province_index|Date1_index|
+----+----+--------+---+----+-----------+-----------+------------+----------+-----+------------+--------------+-----------+
2020| 22|22/01/20| 1| 61|Los Angeles|34.30828379|-118.2282411|California| 0| 3.0| 0.0| 84.0|
2020| 23|23/01/20| 1| 59|Los Angeles|34.30828379|-118.2282411|California| 0| 3.0| 0.0| 29.0|
2020| 24|24/01/20| 1| 63|Los Angeles|34.30828379|-118.2282411|California| 0| 3.0| 0.0| 26.0|
2020| 25|25/01/20| 1| 64|Los Angeles|34.30828379|-118.2282411|California| 0| 3.0| 0.0| 105.0|
2020| 26|26/01/20| 1| 58|Los Angeles|34.30828379|-118.2282411|California| 1| 3.0| 0.0| 106.0|
+----+----+--------+---+----+-----------+-----------+------------+----------+-----+------------+--------------+-----------+
only showing top 5 rows

In [5]:
splits = data.randomSplit([0.7, 0.3])
# for gradient boosted tree regression
dt_train = splits[0]
dt_test = splits[1].withColumnRenamed("label", "trueLabel")

print ("Training Rows:", dt_train.count(), " Testing Rows:", dt_test.count())

dt_train.show(5)


Training Rows: 323 Testing Rows: 113
+----+----+--------+---+----+------------+-----------+------------+------------+-----+------------+--------------+-----------+
Year|Date| Date1|Day|Temp| Admin2| Lat| Long| Province|label|Admin2_index|Province_index|Date1_index|
+----+----+--------+---+----+------------+-----------+------------+------------+-----+------------+--------------+-----------+
2020| 1|01/03/20| 3| 44| New York| 40.7672726|-73.97152637| New York| 0| 1.0| 3.0| 21.0|
2020| 1|01/03/20| 3| 47| Spokane|47.62113146|-117.4046494| Washington| 0| 0.0| 1.0| 21.0|
2020| 1|01/03/20| 3| 60|Philadelphia|40.00338507| -75.1379271|Pennsylvania| 0| 2.0| 2.0| 21.0|
2020| 1|01/04/20| 4| 56| New York| 40.7672726|-73.97152637| New York|47439| 1.0| 3.0| 11.0|
2020| 1|01/04/20| 4| 57| Los Angeles|34.30828379|-118.2282411| California| 3518| 3.0| 0.0| 11.0|
+----+----+--------+---+----+------------+-----------+------------+------------+-----+------------+--------------+-----------+
only showing top 5 rows

In [6]:

#assembler = VectorAssembler(inputCols =["Day","Temp","Lat","Long","Admin_index","Province_index"],outputCol="features")
assembler = VectorAssembler(inputCols =["Date","Year","Day","Temp","Admin2_index"],outputCol="features")
#assembler = VectorAssembler(inputCols =["Date","Year","Day","Temp"],outputCol="features")

dt = DecisionTreeRegressor(featuresCol='features', labelCol='label', maxBins=77582)
dt_pipeline = Pipeline(stages=[assembler, dt])
paramGrid = ParamGridBuilder().build()
cv = CrossValidator(estimator=dt_pipeline, evaluator=RegressionEvaluator(), estimatorParamMaps=paramGrid, numFolds=5)
dt_model = cv.fit(dt_train)
dt_prediction = dt_model.transform(dt_test)
dt_predicted = dt_prediction.select("features", "prediction", "trueLabel")
dt_predicted.show(10)

dt_evaluator = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
dt_rmse = dt_evaluator.evaluate(dt_prediction)


print ("Root Mean Square Error (RMSE):", dt_rmse)
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluation = MulticlassClassificationEvaluator(
    labelCol="trueLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluation.evaluate(dt_prediction)
print(accuracy)

+--------------------+------------------+---------+
 features| prediction|trueLabel|
+--------------------+------------------+---------+
[1.0,2020.0,2.0,4...| 34.65384615384615| 0|
[1.0,2020.0,2.0,4...| 0.0| 0|
[1.0,2020.0,2.0,5...| 0.0| 0|
[1.0,2020.0,2.0,6...| 1.0| 1|
[1.0,2020.0,3.0,5...|17.736842105263158| 1|
[1.0,2020.0,4.0,4...| 201.0| 158|
[1.0,2020.0,4.0,5...| 7122.878787878788| 1675|
[1.0,2020.0,5.0,6...|27908.333333333332| 24262|
[2.0,2020.0,2.0,4...| 0.0| 0|
[2.0,2020.0,2.0,4...| 0.0| 0|
+--------------------+------------------+---------+
only showing top 10 rows

Root Mean Square Error (RMSE): 4278.315341082537
0.35398230088495575